In [9]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Activation,Dropout,Input
from sklearn import cross_validation
import numpy as np

# Read data from store.h5

In [2]:
import pandas as pd
store = pd.HDFStore('store.h5')

# print the item in store

In [12]:
print store.items
data=store['onehot_data']
label=store['label']
selected_idx=store['selected_idx']

<bound method HDFStore.items of <class 'pandas.io.pytables.HDFStore'>
File path: store.h5
/data                             frame        (shape->[1000,9443]) 
/feature_score                    series       (shape->[28329])     
/feature_selected_data            frame        (shape->[1000,3169]) 
/gene_info                        frame        (shape->[300,9443])  
/label                            frame        (shape->[1000,1])    
/onehot_data                      frame        (shape->[1000,28329])
/selected_idx                     series       (shape->[28329])     >


In [36]:
X = data.as_matrix()
X_new2 = X[:,selected_idx]

/home/dl/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  from ipykernel import kernelapp as app


# Close the store.h5

In [ ]:
store.close()

In [ ]:
data.head()

# seperate the data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(label)
target=enc.transform(label).toarray()

## use sklearn.cross_validation.train_test_split

In [ ]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(data,target, test_size=0.3, random_state=0)

# Use Auto-Encoder

In [ ]:
input_img = Input(shape=(X_train.shape[1],))
encoded = Dense(8000, activation='relu')(input_img)
encoded = Dense(2000, activation='relu')(encoded)
encoded = Dense(500, activation='relu')(encoded)

decoded = Dense(2000, activation='relu')(encoded)
decoded = Dense(8000, activation='relu')(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)

autoencoder = Model(input=input_img, output=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(X_train.values,X_train.values,
                verbose=1,
                nb_epoch=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test.values, X_test.values))

In [ ]:
model=Sequential()

In [ ]:
print autoencoder.get_weights()[0].shape
print autoencoder.get_weights()[1].shape
print autoencoder.get_weights()[2].shape
print autoencoder.get_weights()[3].shape
print autoencoder.get_weights()[4].shape
print autoencoder.get_weights()[5].shape
print autoencoder.get_weights()[6].shape
print autoencoder.get_weights()[7].shape
print autoencoder.get_weights()[8].shape
print autoencoder.get_weights()[9].shape
print autoencoder.get_weights()[10].shape
print autoencoder.get_weights()[11].shape

# Use Keras to build the Nets
[None,28329]->[None,8000]->[None,2000]->[None,500]->[2]

In [ ]:
model = Sequential()
model.add(Dense(8000, input_dim=X_train.shape[1],weights=[autoencoder.get_weights()[0],autoencoder.get_weights()[1]],activation='relu'))

model.add(Dense(2000,input_dim=8000,weights=[autoencoder.get_weights()[2],autoencoder.get_weights()[3]],activation='relu'))

model.add(Dense(500,input_dim=2000,weights=[autoencoder.get_weights()[4],autoencoder.get_weights()[5]],activation='relu'))



model.add(Dense(2,input_dim=500,activation='sigmoid'))


model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train.values,y_train,batch_size=100,nb_epoch=500,verbose=1,validation_split=0.2)